In [1]:
import tensorflow as tf
# import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

class Model:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self):
        with tf.variable_scope(self.name):
            self.keep_prob = tf.placeholder(tf.float32)
            
            self.X = tf.placeholder(tf.float32, [None, 28*28])
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])
            
            W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
            L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding='SAME')
            L1_relu = tf.nn.relu(L1)
            L1_mp = tf.nn.max_pool(L1_relu, ksize=[1,2,2,1],
                                  strides=[1,2,2,1], padding='SAME')
            L1_dp = tf.nn.dropout(L1_mp, self.keep_prob)
            
            W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01))
            L2 = tf.nn.conv2d(L1_dp, W2, strides=[1,1,1,1], padding='SAME')
            L2_relu = tf.nn.relu(L2)
            L2_mp = tf.nn.max_pool(L2_relu, ksize=[1,2,2,1],
                                strides=[1,2,2,1], padding='SAME')
            L2_dp = tf.nn.dropout(L2_mp, self.keep_prob)
            
            W3 = tf.Variable(tf.random_normal([3,3,64,128], stddev=0.01))
            L3 = tf.nn.conv2d(L2_dp, W3, strides=[1,1,1,1], padding='SAME')
            L3_relu = tf.nn.relu(L3)
            L3_mp = tf.nn.max_pool(L3_relu, ksize=[1,2,2,1],
                                  strides=[1,2,2,1], padding='SAME')
            L3_dp = tf.nn.dropout(L3_mp, self.keep_prob)
            
            L3_flat = tf.reshape(L3_dp, [-1, 128*4*4])
            
            W4 = tf.get_variable('W4', shape = [4*4*128, 625],
                                initializer = tf.contrib.layers.xavier_initializer())
            #W4 = tf.Variable(tf.random_normal([4*4*128, 625]))
            b4 = tf.Variable(tf.random_normal([625]))
            L4 = tf.nn.relu(tf.matmul(L3_flat, W4) + b4)
            L4_dp = tf.nn.dropout(L4, self.keep_prob)
            
            W5 = tf.get_variable("W5", shape=[625, 10],
                                initializer=tf.contrib.layers.xavier_initializer())
            b5 = tf.Variable(tf.random_normal([10]))
            self.logits = tf.matmul(L4_dp, W5) + b5
        
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.loss)
        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    def predict(self, x_test, keep_prob=1.0):
        return self.sess.run(self.logits, {self.X: x_test, self.keep_prob: keep_prob})
    
    def get_acc(self, x_test, y_test, keep_prob=1.0):
        return self.sess.run(self.acc, {self.X: x_test, self.Y: y_test, self.keep_prob: keep_prob})
    
    def train(self, x_data, y_data, keep_prob=0.7):
        return self.sess.run([self.optimizer, self.loss], {
            self.X: x_data, self.Y: y_data, self.keep_prob: keep_prob})

sess = tf.Session()
m1 = Model(sess, "m1")

sess.run(tf.global_variables_initializer())

print('Learning started.')

for epoch in range(training_epochs):
    avg_loss = 0
    total_batch = int(mnist.train.num_examples/batch_size)
    
    for i in range(total_batch):
        x, y = mnist.train.next_batch(batch_size)
        _, l = m1.train(x, y)
        avg_loss += l/total_batch
        
    print('Epoch:', '%02d' % (epoch+1), 'loss = ', '{:.9f}'.format(avg_loss))
print('Learning ended.')
print('Accuracy:', m1.get_acc(mnist.test.images, mnist.test.labels))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started.
Epoch: 01 loss =  0.371243244
Epoch: 02 loss =  0.100204814
Epoch: 03 loss =  0.075080394
Epoch: 04 loss =  0.061391210
Epoch: 05 loss =  0.055095142
Epoch: 06 loss =  0.047679551
Epoch: 07 loss =  0.042886105
Epoch: 08 loss =  0.040747744
Epoch: 09 loss =  0.036955991
Epoch: 10 loss =  0.035242880
Epoch: 11 loss =  0.033533396
Epoch: 12 loss =  0.030518434
Epoch: 13 loss =  0.030822504
Epoch: 14 loss =  0.027282690
Epoch: 15 loss =  0.026255302
Learning ended.
Accuracy: 0.9938
